# Wordle Bot
Automatyzacja gry w Wordle po angielsku (`https://wordly.org`). 

In [2]:
import selenium as se
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pandas import DataFrame, read_csv
import numpy as np

In [16]:
driver=webdriver.Firefox()
driver.get("https://wordly.org")

Pozbycie się okienka odnośnie zbierania danych

In [17]:
manage_option=driver.find_element(by=By.CLASS_NAME, value="fc-cta-manage-options")
manage_option.click()
confirm_choices=driver.find_element(by=By.CLASS_NAME, value="fc-confirm-choices")
confirm_choices.click()
#TODO:
# - dodać odrzucenie wyborów


Załadowanie listy słów

In [ ]:
word_list=read_csv("./valid-words.csv") # valid words ma więcej liter, sprawdzić czy word bank styknie
word_list=word_list.values.flatten()
print(word_list)
print(len(word_list))

['aalii' 'aargh' 'aarti' ... 'zygon' 'zymes' 'zymic']
12971


# Strategia początkowa - CRANE

In [28]:
turn=0

Wpisanie słowa

In [51]:
input_word=word_list[0]
keyboard=driver.find_element(by=By.CLASS_NAME, value="Game-keyboard")
letter_buttons=keyboard.find_elements(by=By.CLASS_NAME, value="Game-keyboard-button")
enter_button=None
for button in letter_buttons:
    if button.text=="Enter":
        enter_button=button
if enter_button is None:
    raise ValueError
for letter in input_word:
    print(letter)
    for button in letter_buttons:
        if button.text.lower()==letter:
            print(button.rect)
            button.click()
enter_button.click()


q
{'x': 343.33331298828125, 'y': 460.0, 'width': 54.0, 'height': 46.0}
u
{'x': 703.3333129882812, 'y': 460.0, 'width': 54.0, 'height': 46.0}
e
{'x': 463.3333129882813, 'y': 460.0, 'width': 54.0, 'height': 46.0}
u
{'x': 703.3333129882812, 'y': 460.0, 'width': 54.0, 'height': 46.0}
e
{'x': 463.3333129882813, 'y': 460.0, 'width': 54.0, 'height': 46.0}


Przechwycenie wyniku

In [48]:
placement_value={
    "letter-absent":0,
    "letter-elsewhere":1,
    "letter-correct":2
}
rows=driver.find_element(by=By.CLASS_NAME,value="game_rows").find_elements(by=By.CLASS_NAME,value="Row")
print(rows[turn].rect)
info=[]
row_letters=rows[turn].find_elements(by=By.CLASS_NAME,value="Row-letter")
for row_letter in row_letters:
    letter=row_letter.text.lower()
    placement_class=row_letter.get_attribute("class").split(" ")[1]
    info.append((letter,placement_value[placement_class]))
print(info)

    
    


{'x': 340.33331298828125, 'y': 303.0, 'width': 600.0, 'height': 62.0}
[('q', 2), ('u', 2), ('e', 2), ('m', 0), ('e', 2)]


Filtrowanie listy słów
TODO:
- filtracja listy słów
- sprawdzenie czy jest koniec gry (div class="message")

In [49]:
def is_word_correct(word:str,info_list:list[tuple[str,int]])->bool:
    for index,i in enumerate(info_list):
        if i[1]==0:
            for char in word:
                if char==i[0].lower():
                    return False
        elif i[1]==1:
            res=False
            for j,char in enumerate(word):
                if index==j and char==i[0].lower():
                    return False
                if index!=j and char==i[0].lower():
                    res=True
            if not res:
                return False
        else:
            if word[index]!=i[0].lower():
                return False
    return True
                
#TODO:
# uwzględnij duplikaty w literach

In [50]:
new_list=[]
for word in word_list:
    if is_word_correct(word,info):
        new_list.append(word)
word_list=np.array(new_list)
print(new_list)
print(len(new_list))
turn+=1


[np.str_('queue')]
1


powtarzamy aż zwyciężymy